In [1]:
import numpy as np

## Creating Grid and Piece Class

In [2]:
class Grid:
    
    def __init__(self, grid = [[0]*8]*8, colorgrid = [["--"]*8]*8):
        self.grid = np.array(grid, dtype = np.int)
        self.colorgrid = np.array(colorgrid, dtype = object)
        
    def add_piece(self, piece, x, y, orientation_index):
        assert (orientation_index < piece.max_orient), "Orientation exceeds number of orientations"
        new_piece = piece.reorient(piece.orientations[orientation_index])
        xdim = new_piece.grid.shape[0]
        ydim = new_piece.grid.shape[1]
        assert (x + xdim <= 8 and y + ydim <= 8), "Piece is out of bounds"
        self.grid[x:(x + xdim),y:(y + ydim)] += new_piece.grid
        self.colorgrid[x:(x + xdim),y:(y + ydim)] += new_piece.colorgrid
            
    def good_grid(self):
        return(not np.any(self.grid > 1))
    
    def empty_space(self, x, y):
        return(self.grid[x,y] == 0)
    
    def gprint(self):
        s = [[str(e) for e in row] for row in self.grid]
        lens = [max(map(len, col)) for col in zip(*s)]
        fmt = ' '.join('{{:{}}}'.format(x) for x in lens)
        table = [fmt.format(*row) for row in s]
        print('\n'.join(table))
            
    def cprint(self):
        s = [[str(e)[(len(str(e)) - 2):] for e in row] for row in self.colorgrid]
        lens = [max(map(len, col)) for col in zip(*s)]
        fmt = ' '.join('{{:{}}}'.format(x) for x in lens)
        table = [fmt.format(*row) for row in s]
        print('\n'.join(table))
        
    def grid_to_string(self):
        s = [[str(e)[(len(str(e)) - 2):] for e in row] for row in self.colorgrid]
        lens = [max(map(len, col)) for col in zip(*s)]
        fmt = ' '.join('{{:{}}}'.format(x) for x in lens)
        table = [fmt.format(*row) for row in s]
        return('\n'.join(table))

In [3]:
class Piece:
    
    def __init__(self, grid, color):
        self.grid = np.array(grid, dtype = np.int)
        self.color = color
        self.colorgrid = np.zeros(self.grid.shape, dtype = object)
        self.colorgrid[self.grid == 1] = self.color
        self.colorgrid[self.grid == 0] = ""
        self.__rotsym = np.array_equal(self.grid, np.rot90(self.grid, k = 2))
        self.__flipsym = np.array_equal(self.grid, np.flip(np.rot90(self.grid), axis = 1)) or \
        np.array_equal(self.grid, np.flip(self.grid, axis = 1))
        if self.__rotsym and self.__flipsym:
            self.max_orient = 2
            self.orientations = [0,1]
        elif self.__rotsym:
            self.max_orient = 4
            self.orientations = [0,1,4,5]
        elif self.__flipsym:
            self.max_orient = 4
            self.orientations = [0,1,2,3]
        else:
            self.max_orient = 8
            self.orientations = [0,1,2,3,4,5,6,7]
            
    def reorient(self, orientation):
        assert (orientation in self.orientations), "Not a valid orientation for this piece"
        # rotate piece appropriate number of rotations
        gr = np.rot90(self.grid, k = orientation % 4)
        # flip piece if needed
        if orientation > 3:
            gr = np.flip(gr, axis = 1)
        # add piece to grid in correct spot
        return Piece(gr.tolist(), self.color)

## Creating Pieces

In [4]:
BLUE = Piece([[1,1,1,0,0],[0,0,1,0,0],[0,0,1,1,1]], "bl")
BROWN = Piece([[1,1,1,1],[1,1,1,1]], "br")
ORANGE = Piece([[1,0,0],[1,0,0],[1,1,1]], "or")
DARKGREEN = Piece([[1,1,0],[1,1,1],[1,1,1]], "dg")
YELLOW = Piece([[1,0,0],[1,0,0],[1,0,0],[1,1,1]], "ye")
PINK = Piece([[1,1],[1,1],[1,0],[1,0]],"pi")
PURPLE = Piece([[1,1,1],[1,1,1],[1,0,0],[1,0,0]], "pu")
LIGHTGREEN = Piece([[1,1],[1,0],[1,0]],"lg")
WHITE = Piece([[1,1],[1,1],[1,0],[1,0],[1,0]], "wh")
AQUA = Piece([[1,1],[1,1],[1,0]], "aq")

PIECES = [BLUE, BROWN, PURPLE, WHITE, DARKGREEN, YELLOW, ORANGE, PINK, LIGHTGREEN, AQUA]

## Building Functions to Run Solutions

In [31]:
def get_solutions(pieces, path):
    file = open(path, "w")
    sol_num = 0
    
    SOLUTIONS = []
    
    def recurse(grid, piece_to_add, x, y, orientation, other_remaining_pieces, k):
        # print("Adding " + piece_to_add.color + " to " + str((x, y)))
        # grid.cprint()
        new_piece = piece_to_add.reorient(piece_to_add.orientations[orientation])
        xdim = new_piece.grid.shape[0]
        ydim = new_piece.grid.shape[1]
        if x + xdim <= 8 and y + ydim <= 8:
            grid.add_piece(piece_to_add, x, y, orientation)
            if grid.good_grid() and not other_remaining_pieces:
                print(". solution found!")
                file.write("Solution " + str(sol_num) + ":\n")
                file.write(grid.grid_to_string() + "\n")
                grid.cprint()
                SOLUTIONS.append(grid)
                sol_num += 1
            elif grid.good_grid():
                next_piece = other_remaining_pieces[0]
                for r in range(8):
                    for c in range(8):
                        if grid.empty_space(r, c):
                            for o in range(next_piece.max_orient):
                                new_grid = Grid(grid.grid, grid.colorgrid)
                                recurse(new_grid, next_piece, r, c, o, other_remaining_pieces[1:], k + 1)
    
    for r in range(8):
        for c in range(8):
            for o in range(pieces[0].max_orient):
                print("Piece 1 in row " + str(r) + ", column " + str(c) + ", orientation " + str(o))
                recurse(Grid(), pieces[0], r, c, o, pieces[1:], 1)
                
    file.close()
    
    return(SOLUTIONS)

In [32]:
solutions = get_solutions(PIECES, "solutions.txt")

Piece 1 in row 0, column 0, orientation 0
. solution found!
bl bl bl pu pu pu br br
wh wh bl pu pu pu br br
wh wh bl bl bl pu br br
wh ye ye ye ye pu br br
wh dg dg dg ye or or or
wh dg dg dg ye or aq aq
pi pi dg dg lg or aq aq
pi pi pi pi lg lg lg aq
. solution found!
bl bl bl pu pu pu br br
wh wh bl pu pu pu br br
wh wh bl bl bl pu br br
wh ye ye ye ye pu br br
wh dg dg dg ye aq aq aq
wh dg dg dg ye aq aq or
pi pi dg dg lg lg lg or
pi pi pi pi lg or or or
. solution found!
bl bl bl pu pu pu br br
wh wh bl pu pu pu br br
wh wh bl bl bl pu br br
wh ye ye ye ye pu br br
wh dg dg dg ye or or or
wh dg dg dg ye or aq aq
lg dg dg pi pi or aq aq
lg lg lg pi pi pi pi aq
. solution found!
bl bl bl pu pu pu br br
wh wh bl pu pu pu br br
wh wh bl bl bl pu br br
wh ye ye ye ye pu br br
wh dg dg dg ye aq aq aq
wh dg dg dg ye aq aq or
lg dg dg pi pi pi pi or
lg lg lg pi pi or or or
. solution found!
bl bl bl pu pu pu br br
wh wh bl pu pu pu br br
wh wh bl bl bl pu br br
wh ye ye ye ye pu br br
wh y

. solution found!
bl bl bl pu pu pu br br
aq aq bl pu pu pu br br
aq aq bl bl bl pu br br
wh aq or or or pu br br
wh ye ye ye or dg dg dg
wh lg lg ye or dg dg dg
wh wh lg ye pi pi dg dg
wh wh lg ye pi pi pi pi
. solution found!
bl bl bl pu pu pu br br
pi pi bl pu pu pu br br
pi pi bl bl bl pu br br
wh pi aq aq aq pu br br
wh pi aq aq or or or ye
wh lg lg dg dg dg or ye
wh wh lg dg dg dg or ye
wh wh lg dg dg ye ye ye
. solution found!
bl bl bl pu pu pu br br
pi pi bl pu pu pu br br
pi pi bl bl bl pu br br
wh pi aq aq aq pu br br
wh pi aq aq ye ye ye ye
wh lg lg or ye dg dg dg
wh wh lg or ye dg dg dg
wh wh lg or or or dg dg
. solution found!
bl bl bl pu pu pu br br
ye lg bl pu pu pu br br
ye lg bl bl bl pu br br
ye lg lg wh wh pu br br
ye ye ye wh wh wh wh wh
dg dg aq aq aq or or or
dg dg dg aq aq pi pi or
dg dg dg pi pi pi pi or
. solution found!
bl bl bl pu pu pu br br
pi pi bl pu pu pu br br
pi pi bl bl bl pu br br
pi aq aq wh wh pu br br
pi aq aq wh wh wh wh wh
dg dg aq ye lg or or o

bl bl bl aq aq aq br br
pu pu bl aq aq lg br br
pu pu bl bl bl lg br br
pu pu pu pu lg lg br br
or wh wh wh wh wh pi pi
or wh wh ye dg dg pi pi
or or or ye dg dg dg pi
ye ye ye ye dg dg dg pi
. solution found!
bl bl bl aq aq aq br br
pu pu bl aq aq lg br br
pu pu bl bl bl lg br br
pu pu pu pu lg lg br br
pi dg dg dg ye ye ye ye
pi dg dg dg ye or or or
pi pi dg dg ye wh wh or
pi pi wh wh wh wh wh or
. solution found!
bl bl bl aq aq aq br br
pu pu bl aq aq pi br br
pu pu bl bl bl pi br br
pu pu pu pu pi pi br br
ye or or or pi pi dg dg
ye or lg lg lg dg dg dg
ye or lg wh wh dg dg dg
ye ye ye wh wh wh wh wh
. solution found!
bl bl bl aq aq aq br br
pi pi bl aq aq lg br br
pi pi bl bl bl lg br br
pi pu pu pu lg lg br br
pi pu pu pu ye ye ye ye
or or or pu ye dg dg dg
or wh wh pu ye dg dg dg
or wh wh wh wh wh dg dg
. solution found!
bl bl bl aq aq aq br br
pi pi bl aq aq lg br br
pi pi bl bl bl lg br br
pi pu pu pu lg lg br br
pi pu pu pu ye ye ye ye
dg dg dg pu ye or or or
dg dg dg pu ye w

ye ye ye ye pi dg dg dg
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh pu pu pu
br br bl bl bl pu pu pu
br br dg dg dg pi pi pu
br br dg dg dg pi pi pu
ye lg dg dg or pi aq aq
ye lg lg lg or pi aq aq
ye ye ye ye or or or aq
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh pu pu pu
br br bl bl bl pu pu pu
br br dg dg dg pi pi pu
br br dg dg dg pi pi pu
ye lg dg dg aq aq pi or
ye lg lg lg aq aq pi or
ye ye ye ye aq or or or
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh pu pu pu
br br bl bl bl pu pu pu
br br pi pi dg dg dg pu
br br pi pi dg dg dg pu
or or or pi lg dg dg ye
or aq aq pi lg lg lg ye
or aq aq aq ye ye ye ye
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh pu pu pu
br br bl bl bl pu pu pu
br br pi pi dg dg dg pu
br br pi pi dg dg dg pu
aq aq pi or lg dg dg ye
aq aq pi or lg lg lg ye
aq or or or ye ye ye ye
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh pu pu pu
br br bl bl bl pu pu pu
br br pi pi lg lg lg pu
br br pi pi lg dg dg p

. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh dg dg dg
br br bl bl bl dg dg dg
br br pu pu pu pu dg dg
br br pu pu ye ye ye ye
lg lg pu pu ye or or or
lg aq aq aq ye pi pi or
lg aq aq pi pi pi pi or
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh ye ye ye
br br bl bl bl lg lg ye
br br pu pu pu pu lg ye
br br pu pu aq aq lg ye
dg dg pu pu aq aq aq or
dg dg dg pi pi pi pi or
dg dg dg pi pi or or or
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh aq aq ye
br br bl bl bl aq aq ye
br br pu pu pu pu aq ye
br br pu pu lg ye ye ye
dg dg pu pu lg lg lg or
dg dg dg pi pi pi pi or
dg dg dg pi pi or or or
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh or or or
br br bl bl bl lg lg or
br br pu pu pu pu lg or
br br pu pu aq aq lg ye
dg dg pu pu aq aq aq ye
dg dg dg pi pi pi pi ye
dg dg dg pi pi ye ye ye
. solution found!
bl bl bl ye ye ye ye wh
br br bl lg lg lg ye wh
br br bl bl bl lg ye wh
br br pu pu pu pu wh wh
br br pu pu aq aq wh wh
dg dg pu pu aq aq aq o

...

pu pu pu pu lg lg dg dg
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl ye ye ye
br br wh wh wh wh wh ye
br br or or or wh wh ye
pu pu or dg dg dg aq ye
pu pu or dg dg dg aq aq
pu pu pu pu dg dg aq aq
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl ye ye ye
br br wh wh wh wh wh ye
br br or or or wh wh ye
pu pu or dg dg dg aq ye
pu pu or dg dg dg aq aq
pu pu pu pu dg dg aq aq
. solution found!
bl bl bl lg lg lg pi pi
br br bl lg pi pi pi pi
br br bl bl bl ye ye ye
br br wh wh wh wh wh ye
br br or or or wh wh ye
pu pu or dg dg dg aq ye
pu pu or dg dg dg aq aq
pu pu pu pu dg dg aq aq
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl ye ye ye
br br wh wh wh wh wh ye
br br or or or wh wh ye
pu pu or aq aq dg dg ye
pu pu or aq aq dg dg dg
pu pu pu pu aq dg dg dg
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl ye ye ye
br br wh wh wh wh wh ye
br br or or or wh wh y

pu pu pu pu ye ye ye ye
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl dg dg dg
br br or aq aq dg dg dg
br br or aq aq aq dg dg
pu pu or or or wh wh ye
pu pu wh wh wh wh wh ye
pu pu pu pu ye ye ye ye
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl dg dg dg
br br or aq aq dg dg dg
br br or aq aq aq dg dg
pu pu or or or wh wh ye
pu pu wh wh wh wh wh ye
pu pu pu pu ye ye ye ye
. solution found!
bl bl bl lg lg lg pi pi
br br bl lg pi pi pi pi
br br bl bl bl dg dg dg
br br or aq aq dg dg dg
br br or aq aq aq dg dg
pu pu or or or wh wh ye
pu pu wh wh wh wh wh ye
pu pu pu pu ye ye ye ye
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl dg dg dg
br br or or or dg dg dg
br br or aq aq aq dg dg
pu pu or aq aq wh wh ye
pu pu wh wh wh wh wh ye
pu pu pu pu ye ye ye ye
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl dg dg dg
br br or or or dg dg dg
br br or aq aq aq dg d

. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh lg pi pi
br br bl bl bl lg pi pi
br br or or or lg lg pi
br br or ye ye ye aq pi
dg dg or ye pu pu aq aq
dg dg dg ye pu pu aq aq
dg dg dg ye pu pu pu pu
. solution found!
bl bl bl wh wh wh wh wh
br br bl wh wh lg lg lg
br br bl bl bl aq aq lg
br br or or or aq aq pi
br br or ye ye ye aq pi
dg dg or ye pu pu pi pi
dg dg dg ye pu pu pi pi
dg dg dg ye pu pu pu pu
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl aq aq wh
br br or or or aq aq wh
br br or ye ye ye aq wh
dg dg or ye pu pu wh wh
dg dg dg ye pu pu wh wh
dg dg dg ye pu pu pu pu
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl aq aq wh
br br or or or aq aq wh
br br or ye ye ye aq wh
dg dg or ye pu pu wh wh
dg dg dg ye pu pu wh wh
dg dg dg ye pu pu pu pu
. solution found!
bl bl bl lg lg lg pi pi
br br bl lg pi pi pi pi
br br bl bl bl aq aq wh
br br or or or aq aq wh
br br or ye ye ye aq wh
dg dg or ye pu pu wh w

ye ye ye ye pu pu pu pu
. solution found!
bl bl bl pi pi or or or
br br bl pi pi pi pi or
br br bl bl bl aq aq or
br br dg dg dg aq aq aq
br br dg dg dg lg lg lg
ye wh wh dg dg lg pu pu
ye wh wh wh wh wh pu pu
ye ye ye ye pu pu pu pu
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl or or or
br br dg dg dg aq aq or
br br dg dg dg aq aq or
ye wh wh dg dg aq pu pu
ye wh wh wh wh wh pu pu
ye ye ye ye pu pu pu pu
. solution found!
bl bl bl lg pi pi pi pi
br br bl lg lg lg pi pi
br br bl bl bl or or or
br br dg dg dg aq aq or
br br dg dg dg aq aq or
ye wh wh dg dg aq pu pu
ye wh wh wh wh wh pu pu
ye ye ye ye pu pu pu pu
. solution found!
bl bl bl lg lg lg pi pi
br br bl lg pi pi pi pi
br br bl bl bl or or or
br br dg dg dg aq aq or
br br dg dg dg aq aq or
ye wh wh dg dg aq pu pu
ye wh wh wh wh wh pu pu
ye ye ye ye pu pu pu pu
. solution found!
bl bl bl pi pi lg lg lg
br br bl pi pi pi pi lg
br br bl bl bl or or or
br br aq aq dg dg dg or
br br aq aq dg dg dg o

KeyboardInterrupt: 